In [1]:
import networkx as nx
import pandas as pd
import re
import jieba
import math
#from util import split_sentence
from termcolor import colored, cprint



In [2]:

def split_sentence(text):
    #remove strange character
    text = text.replace("\r\n","")
    #split sentences
    sentences = re.split('([。?!！？.])', text)
    new_sentences = []
    for i in range(len(sentences)//2):
        sent = sentences[2*i] + sentences[2*i+1]
        new_sentences.append(sent)
    return new_sentences
def get_stop_words(stopwords_file):
    stop_words = []
    for line in open(stopwords_file, 'r', encoding='utf-8'):
        stop_words.append(line.replace('\n', ''))
    return stop_words

#Chinese
def get_token(sent):
    #remove punctuation for Chinese
    sent = re.findall(r'[\d|\w]+', sent)
    stopwords = get_stop_words("stopwords.txt")
    #Chinese
    words = list(jieba.cut("".join(sent)))
    #remove stopwords
    words = list(set(words) - set(stopwords))
    return words

#English
def get_token(sent):

    stopwords = get_stop_words("stopwords.txt")

    words = list(sent.split())
    #remove stopwords
    words = list(set(words) - set(stopwords))
    return words

In [3]:
data = pd.read_csv("../sqlResult_1558435.csv", encoding='gb18030')

In [4]:
data.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [15]:
def get_summarization_by_textrank(text = None, title = None, summary_ratio = None):
    """
    text : text body of news
    title: title of news
    summary ratio : ratio of sentence number between text body and summary
    """
    if text == None:
        print("please input text !")
        return None
    #get sentence ranking
    ranking_sentences_id, sentences = sentences_ranking(text, title)
    
    print("The number of text : ", len(ranking_sentences_id))
    
    sentence_candidates = ranking_sentences_id[:len(sentences)//summary_ratio]
    
    print("The number of summarization: ", len(sentence_candidates))
    #print(sentence_candidates)
    #print(sentences)
    return " ".join([sentences[s[0]] for s in sorted(sentence_candidates)])
    #text = colored('Hello, World!', 'red', attrs=['reverse', 'blink'])+'hello'
    #index_list = [s[0] for s in sorted(sentence_candidates)]
    #text = ""
    #for ii, s in enumerate(sentences):
    #    if ii in index_list:
    #        text += colored(sentences[ii], 'red', attrs=['reverse', 'blink'])
    #    else:
    #        text += sentences[ii]
    #return text
        

In [6]:
def sentences_ranking(text, title):
    #get sentences graph
    sentences, sentences_graph, title_flag = get_connect_graph_by_weight_text_rank(text, title)
    #get sentence rank
    ranking_sentences = nx.pagerank(sentences_graph)

    #sort except for title
    if title_flag:
        ranking_sentences.pop(0)

    sorted_ranking_sentences = sorted(ranking_sentences.items(), key = lambda x:x[1], reverse = True)
    return sorted_ranking_sentences, sentences

In [7]:
def get_connect_graph_by_weight_text_rank(text, title):
    """
    text : text body for one news
    title: title for one news
    """
    #text preprocess
    sentences = split_sentence(text)
    tokens = [get_token(s) for s in sentences]
    #title preprocess
    title_flag = False
    #if title:
    #    title_token = get_token(title)
    #    if title_token:
    #        title_flag = True
    #        tokens.insert(0, title_token)
    #        sentences.insert(0, title)
    #add edges
    sentences_graph = nx.Graph()
    for ii, sentence in enumerate(tokens):
        for i in range(ii+1, len(tokens)):
            num_same_words = len(set(tokens[ii]).intersection(set(tokens[i])))
            #double weight for first sentence
            if ii == 0:
                similarity = 2 * num_same_words / (math.log(len(tokens[ii])) + math.log(len(set(tokens[i])))) 
            else:
                similarity = num_same_words / (math.log(len(tokens[ii])) + math.log(len(set(tokens[i]))))
            
            sentences_graph.add_edges_from([(ii, i)],weight = similarity)
    return sentences, sentences_graph, title_flag

In [16]:
if __name__ == "__main__":
    
    text = "网易娱乐7月21日报道林肯公园主唱查斯特·贝宁顿 Chester Bennington于今天早上,在洛杉矶帕洛斯弗迪斯的一个私人庄园自缢身亡,年仅41岁。此消息已得到洛杉矶警方证实。洛杉矶警方透露, Chester的家人正在外地度假, Chester独自在家,上吊地点是家里的二楼。一说是一名音乐公司工作人员来家里找他时发现了尸体,也有人称是佣人最早发现其死亡。林肯公园另一位主唱麦克信田确认了 Chester Bennington自杀属实,并对此感到震惊和心痛,称稍后官方会发布声明。Chester昨天还在推特上转发了一条关于曼哈顿垃圾山的新闻。粉丝们纷纷在该推文下留言,不相信 Chester已经走了。外媒猜测,Chester选择在7月20日自杀的原因跟他极其要好的朋友Soundgarden(声音花园)乐队以及AudioslaveChris乐队主唱 Cornell有关,因为7月20日是 Chris CornellChris的诞辰。而 Cornell于今年5月17日上吊自杀,享年52岁。 Chris去世后, Chester还为他写下悼文。对于 Chester的自杀,亲友表示震惊但不意外,因为 Chester曾经透露过想自杀的念头,他曾表示自己童年时被虐待,导致他医生无法走出阴影,也导致他长期酗酒和嗑药来疗伤。目前,洛杉矶警方仍在调查Chester的死因。据悉, Chester与毒品和酒精斗争多年,年幼时期曾被成年男子性侵,导致常有轻生念头。 Chester生前有过2段婚姻,育有6个孩子。林肯公园在今年五月发行了新专辑《多一丝曙光OneMoreLight》,成为他们第五张登顶ilboard排行榜的专辑。而昨晚刚刚发布新单《 Talking To Myself》MV"
    #text = data["content"][6]
    #title = data["title"][4]#
    text = "I took EECS 545 machine learning class last semester. I implement different kinds of supervised machine learning algorithm step by step in python. For example, kNN classifier, Regularized  Linear  Regression, Weighted Linear Regression, logistic regression, ridge regression, Support Vector Regression, Bayesian, Gradient and Coordinate Decent and Neural Nets. And some supervised machine learning algorithm, like LDA, QDA, K-means, pca. Thus, I am not only familiar how to use machine learning model from some packages, like sklearn, but also know about how to implement in python and mathematic  deduction behind different model. In addition, I have some deep learning experiences with machine learning framework, like PyTorch and Tensorflow. "
    title = None

    
    text = "IT HAS BECOME A RECENT COMMONPLACE to recognize that regions, like nations, are imagined constructs, that they possess historical specificity and are imagined differently by different people at various times.1 It is less well acknowledged, however, that there are also flexible imaginaries that perhaps are related to and extend already existing structures, or that turn against these structures and contest their claims.2 These flexible imaginaries require looking at specific moments and episodes that have either been repressed by state-centered narratives of history or displaced by nostalgic yearnings for the local in an environment of dominating global and total structures. This essay focuses on a moment in early twentieth-century China during which Asia was first created as a regional construct by a small but influential group of Chinese intellectuals, who, in collaboration with other peoples residing in Tokyo, attempted to rethink modes of political action and organization. The Asia that resulted from these activities and rethinkings was shaped by, albeit not limited to, two simultaneous structures: imperialism and ongoing attempts to subject the non-West to a Western-dominated world system; and state-dominated concepts of national and regional formation. These two structural trends helped produce a concept of Asia that is well known to historians: an anti-imperialist and yet state-centered regional formation that has episodically resulted in various state-led pan-Asianisms over the course of the ensuing century. Sun Zhongshan's (Sun Yat-sen) state-based, anti-imperialist vision of Asia is the most familiar of these versions. However, a different discourse of Asia was also produced at this time, one that was rooted in non-state-centered practices and in non-national-chauvinist cultur- alism. This alternative Asia discourse and practice represented an attempt to create a radically politicized cultural regional concept, whose historical resonance seems, however, to have been almost permanently displaced by the more strident versions of the region that uphold the primacy of the nation-state and capitalist developmentalism."
    summary_ratio = 3
    summarization = get_summarization_by_textrank(text, title, summary_ratio)
    print(summarization)

The number of text :  9
The number of summarization:  3
[(5, 0.13963011201012046), (8, 0.1343225054735141), (0, 0.13279828696284351)]
['IT HAS BECOME A RECENT COMMONPLACE to recognize that regions, like nations, are imagined constructs, that they possess historical specificity and are imagined differently by different people at various times.', '1 It is less well acknowledged, however, that there are also flexible imaginaries that perhaps are related to and extend already existing structures, or that turn against these structures and contest their claims.', '2 These flexible imaginaries require looking at specific moments and episodes that have either been repressed by state-centered narratives of history or displaced by nostalgic yearnings for the local in an environment of dominating global and total structures.', ' This essay focuses on a moment in early twentieth-century China during which Asia was first created as a regional construct by a small but influential group of Chinese in

In [9]:
def blueprint(x):
    print('\033[34m' + str(x) + '\033[0m'+"world")
blueprint("Hello")

Helloworld


In [10]:
from colorama import Fore, Back, Style
print('and in dim text'+Back.GREEN + 'and with a green background')
print(Back.GREEN + 'and with a green background')


and in dim textand with a green background
and with a green background


In [11]:
import sys
from termcolor import colored, cprint

text = colored('Hello, World!', 'light_red', attrs=['reverse', 'blink'])+colored('Hello, World!', 'red', attrs=['reverse', 'blink'])
print(text)
cprint('Hello, World!', 'green', 'on_red')

KeyError: 'light_red'

In [ ]:
import sys
from termcolor import colored, cprint

text = colored('Hello, World!', 'red', attrs=['reverse', 'blink'])+'hello'
print(text)
cprint('Hello, World!', 'green', 'on_red')

print_red_on_cyan = lambda x: cprint(x, 'red', 'on_cyan')
print_red_on_cyan('Hello, World!')
print_red_on_cyan('Hello, Universe!')

for i in range(10):
    cprint(i, 'magenta', end=' ')

cprint("Attention!", 'red', attrs=['bold'], file=sys.stderr)

In [ ]:
webcolors.name_to_rgb('red')

In [ ]:
import webcolors